# AutoGen ツール使用例

## 必要なパッケージのインポート

In [2]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## クライアントの作成

このサンプルでは、LLMへのアクセスに[GitHub Models](https://aka.ms/ai-agents-beginners/github-models)を使用します。

`model`は`gpt-4o-mini`として定義されています。GitHub Modelsマーケットプレイスで利用可能な他のモデルに変更して、異なる結果を確認してみてください。

簡単なテストとして、シンプルなプロンプト「フランスの首都は何ですか？」を実行します。

In [3]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # モデルでの認証には、GitHubの設定で個人アクセストークン（PAT）を生成する必要があります。
    # PATトークンの作成手順：https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="フランスの首都は何ですか？", source="user")])
print(result)

c:\Users\Mamoru Kuroda\AppData\Local\Programs\Python\Python313\Lib\site-packages\autogen_ext\models\azure\_azure_ai_client.py:305: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(config["model_info"])


finish_reason='stop' content='フランスの首都はパリです。' usage=RequestUsage(prompt_tokens=18, completion_tokens=12) cached=False logprobs=None thought=None


## 関数の定義

この例では、エージェントに利用可能なバケーション先とその空き状況のリストを持つ関数へのアクセスを提供します。

これは、旅行エージェントが旅行データベースにアクセスするようなシナリオを想定しています。

このサンプルを進める際に、エージェントが呼び出せる新しい関数やツールを自由に定義してみてください。

In [4]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    特定のバケーション先が利用可能かどうかを確認します
    
    Args:
        city (str): 確認する都市名
        
    Returns:
        tuple: 都市名と空き状況（'利用可能'または'利用不可'）を含む
    """
    destinations = {
        "バルセロナ": "利用可能",
        "東京": "利用不可",
        "ケープタウン": "利用可能",
        "バンクーバー": "利用可能",
        "ドバイ": "利用不可",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "都市が見つかりません"

# 使用例：
# city, status = vacation_destinations("バルセロナ")
# print(f"{city}を訪問するのはいかがですか？現在{status}です！")

## 関数ツールの定義

エージェントが`vacation_destinations`を`FunctionTool`として使用するには、それを定義する必要があります。

また、ツールの説明も提供します。これは、ユーザーが要求したタスクに関連してそのツールが何に使用されるかをエージェントが識別するのに役立ちます。

In [5]:
get_vacations = FunctionTool(
    vacation_destinations, description="バケーション先を検索し、利用可能かどうかを確認します。"
)

## エージェントの定義

下記のコードでエージェントを作成できます。`system_message`を定義して、ユーザーがバケーション先を見つけるのを支援する方法についてエージェントに指示を与えます。

また、`reflect_on_tool_use`パラメータをtrueに設定します。これにより、LLMがツール呼び出しの応答を受け取り、自然言語を使用してレスポンスを送信できます。

このパラメータをfalseに設定して違いを確認することもできます。

In [6]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="あなたはユーザーがバケーション先を見つけるのを支援する旅行エージェントです。",
    reflect_on_tool_use=True,
)

## エージェントの実行

東京への旅行を希望する初期ユーザーメッセージでエージェントを実行できます。

この都市の目的地を変更して、エージェントがその都市の空き状況にどのように応答するかを確認できます。

In [7]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="東京に旅行したいのですが", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# スクリプトで実行する場合は asyncio.run(assistant_run()) を使用してください。
await assistant_run()

[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=91, completion_tokens=18), metadata={}, content=[FunctionCall(id='call_PVljbambBPn0nMv7ZG25LXxk', arguments='{"city":"東京"}', name='vacation_destinations')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content="('東京', '利用不可')", name='vacation_destinations', call_id='call_PVljbambBPn0nMv7ZG25LXxk', is_error=False)], type='ToolCallExecutionEvent')]
source='assistant' models_usage=RequestUsage(prompt_tokens=78, completion_tokens=79) metadata={} content='申し訳ありませんが、現在東京に関する具体的な情報は利用できません。ただし、東京への旅行に関してお手伝いできることはたくさんあります！観光名所、食事、宿泊施設、交通手段などについてアドバイスを提供できますので、どのような情報が必要か教えてください。' type='TextMessage'


## 学習のポイント

### AutoGen vs Semantic Kernel の主な違い

- **AutoGen**: エージェント中心のアプローチ。エージェント間の会話や協調作業に重点を置いており、マルチエージェントシステムの構築に適している
- **Semantic Kernel**: カーネル（コア）とプラグイン（ツール）の統合に重点を置いており、単一のAIアシスタントに豊富な機能を追加するのに適している

### このサンプルから学べること

1. **FunctionTool の定義方法**: 既存のPython関数をエージェントが使用できるツールに変換する手法
2. **エージェントの設定**: `reflect_on_tool_use` パラメータによるツール使用後の応答制御
3. **非同期処理**: AutoGenの非同期メッセージング機能の基本的な使用方法

### 次のステップ

- 複数のツールを組み合わせた使用例
- マルチエージェント間でのツール共有
- カスタムツールの開発とテスト